In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import hydra
import settings
import os
os.chdir(settings.current_file_dir)
import torch
from torch.utils.data import DataLoader

import data

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.cluster.hierarchy as sch
from pathlib import Path

from PIL import Image

import torch.nn.functional as F

In [3]:
settings.root

PosixPath('/home/cyanos/Workspace/ood-osa-code/ood-osa-code')

In [4]:
with hydra.initialize(version_base=None, config_path= str('../../..' / settings.hydra_cfg_path.relative_to(settings.root)) ):
    cfg = hydra.compose(config_name="ood_eval", return_hydra_config=True, 
                 overrides=["net=densenet"])

In [5]:
# Log config
DATA_ROOT = Path(os.environ['TORCH_DATASETS'])
print(DATA_ROOT)
if 'data_mean' in cfg.net: # set data mean and std
    data.NORM_PARAMS[cfg.in_dataset]["mean"] = cfg.net.data_mean
    data.NORM_PARAMS[cfg.in_dataset]["std"] = cfg.net.data_std
print(data.NORM_PARAMS[cfg.in_dataset])

# Load in-distribution data
id_data_dict = data.get_id_datasets_dict(DATA_ROOT, cfg.in_dataset)
NUM_CLASSES = id_data_dict['meta']['num_classes']
print(f'Using {data.ID2PRINTNAME[cfg.in_dataset]} as typical data')

# Load ood data
ood_datasets_dict = data.get_ood_datasets_dict(DATA_ROOT, cfg.in_dataset)

/home/cyanos/Workspace/pytorch_datasets
{'mean': [0.4914, 0.4822, 0.4465], 'std': [0.2023, 0.1994, 0.201]}
Using CIFAR-10 as typical data


Dataset was not downloaded /home/cyanos/Workspace/pytorch_datasets/dtd/images
Dataset was not downloaded /home/cyanos/Workspace/pytorch_datasets/places365


In [12]:
ood_data = ood_datasets_dict['ds']['far']['iSUN']

In [15]:
image, _ = ood_data[0]